In [7]:
import pandas as pd
import geopandas as gp
import shapely
from shapely.geometry import Point, LineString, MultiLineString
from shapely.ops import transform
import pyproj
from collections import Counter
import arcpy
arcpy.env.overwriteOutput = True
import os

In [8]:
from lrs_tools import gp_lrs

In [9]:
import logging
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG) # Set the debug level here
fileHandler = logging.FileHandler(f'test.log', mode='w')
log.addHandler(fileHandler)

In [10]:
import importlib
importlib.reload(gp_lrs)

<module 'lrs_tools.gp_lrs' from 'c:\\Users\\daniel.fourquet\\Documents\\Tasks\\TMC Conflation 2025\\NPMRDS\\lrs_tools\\gp_lrs.py'>

#### Description of steps ####

* Round 1:
    * Dissolve TMCs into larger groups by TmcLinear field.  These groups span large portions of a single route, so identifying the route associated with the TmcLinear group, then assigning that route to each member of the TmcLinear group will speed up the process since each TMC will not need to be processed individually.  This will likely result in errors in some individual TMCs, but the errors will be caught at the end of Round 1 and these TMCs will be processed individually.  The routes will be identified by finding the closest route to the begin, mid, and end point of the group and returning the route if only one is counted 3 times.
    * For every TMC whose route was not identified in the step above, locate routes using the begin, mid, and end points of each TMC and returning the route if only one is counted 3 times.
    * For remaining TMCs, identify routes by segment.  A sample of segments from the input TMC are compared to the nearest segment in the LRS.  If the bearing is similar between the two segments, then this route is taken into consideration as a match.
    * If more than one route is identified as a potential match, then a new record will be crated for each match
    * The m-values for each match are identified
    * Routes are flipped to non-prime side if the begin_msr value is larger than the end_msr value
    * Round 1 is exported as a table and feature class in the intermediate geodatabase
    * Round 1 QC

* Round 2 - For segments that fail the Round 1 QC:
    * Run 'detailed' segment-based route identification.  Instead of a sample of segments being used to identify nearby routes, all segments will be used.  This takes longer, but it does a better job of identifying matching routes in cases where there are a lot of nearby routes.
    * Round 2 QC

* Round 3 - For segments that fail the Round 2 QC:
    * Locate and return the m-values for each nearby segment.  

In [11]:
# Create intermediate and output gdb
output_path = r'C:\Users\daniel.fourquet\Documents\Tasks\TMC Conflation 2025\NPMRDS'

intermediate_gdb = f"{output_path}\\intermediate.gdb"
output_gdb = f"{output_path}\\output.gdb"

# Create gdbs if do not exist
for gdb_path in [intermediate_gdb, output_gdb]:
    if os.path.exists(os.path.dirname(gdb_path)):
        if not os.path.exists(gdb_path):
            arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
    else:
        raise Exception(f'Path for GDB does not exist: \n{os.path.dirname(gdb_path)}')

In [31]:
# Load lrs to an LRS object
lrs_path = r'C:\Users\daniel.fourquet\Documents\Tasks\TMC Conflation 2025\Data\master_lrs.shp'

# Filter to only include NB and EB routes, excluding PA and Interstates.  Exclude ramps
lrs = gp_lrs.LRS(lrs_path, filter=True, ramps=0)

# Load lrs overlap to an LRS object
lrs_overlap_path = r'C:\Users\daniel.fourquet\Documents\Tasks\TMC Conflation 2025\Data\overlap_lrs.shp'

# Filter to only include NB and EB routes, excluding PA and Interstates.  Exclude ramps
lrs_overlap = gp_lrs.LRS(lrs_overlap_path, filter=True, ramps=0)


In [66]:
# Load TMCs to a GeoDataFrame
tmcs_path = r'C:\Users\daniel.fourquet\Documents\Tasks\TMC Conflation 2025\NPMRDS\data\NPMRDS_NOVA_Test.shp'
tmcs = gp.read_file(tmcs_path)
tmcs = tmcs[['Tmc', 'RoadNumber', 'RoadName', 'TmcLinear', 'geometry']]
tmcs = tmcs.to_crs(epsg=3968)

# Calculate begin, end, and mid-points
def interpolate(geom, pct):
    return geom.interpolate(pct, normalized=True)
tmcs['begin_point'] = tmcs.geometry.apply(interpolate, pct=0)
tmcs['mid_point'] = tmcs.geometry.apply(interpolate, pct=0.5)
tmcs['end_point'] = tmcs.geometry.apply(interpolate, pct=1)

# Get TMCs dissolved by lineartmc
linear_tmcs = tmcs.dissolve(['TmcLinear', 'RoadName']).reset_index()
linear_tmcs['geometry'] = shapely.line_merge(linear_tmcs.geometry)
linear_tmcs = linear_tmcs.explode()
linear_tmcs['begin_point'] = linear_tmcs.geometry.apply(interpolate, pct=0)
linear_tmcs['mid_point'] = linear_tmcs.geometry.apply(interpolate, pct=0.5)
linear_tmcs['end_point'] = linear_tmcs.geometry.apply(interpolate, pct=1)


c:\ArcGIS_Python\arcgispro-py3-fourquet\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
c:\ArcGIS_Python\arcgispro-py3-fourquet\lib\site-packages\geopandas\geoseries.py:924: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if s.type.startswith("Multi") or s.type == "GeometryCollection":


In [67]:
# Get most common routes near linear tmcs.  Only return results with one match
def match_routes__begin_mid_end_points(tmc, distance=15, point_threshold=3, lrs=lrs):
    log.debug(f'\n{"="*10}')
    log.debug(f'\n{tmc["Tmc"]} - Match Routes by Points - Distance: {distance}')
    
    points = (tmc['begin_point'], tmc['mid_point'], tmc['end_point'])
    
    log.debug('points:')
    for point in points:
        log.debug(f'\t{point}')

    routes = Counter()
    for point in points:
        nearby_routes = gp_lrs.get_nearby_routes(point, distance, lrs)
        routes.update(nearby_routes)

    # Check for no matches
    if len(routes) == 0:
        log.debug('\tNo matches.  Returning None')
        return None

    log.debug(f'\tRoutes:  {routes}')

    # If the number of points that match with a single route is equal
    # to the point_threshold, it is likely a match.  Otherwise a  more
    # precise method is needed.
    most_common_count = routes.most_common(1)[0][1]
    if most_common_count < point_threshold:
        log.debug('\tMost common below threshold.  Returning None')
        return None
    
    most_common_routes = routes.most_common()

    # Find the maximum count
    max_count = most_common_routes[0][1]

    # Filter for elements with the maximum count
    result = [element for element, count in most_common_routes if count == max_count]

    if len(result) == 1:
        log.debug(f'Returning {result[0]}')
        return result[0]
    
    return None


In [68]:
log.debug(f'{"=" * 21}\n       ROUND 1       \n{"=" * 21}')
# Get most common routes
log.debug(f'===\nLinear TMC IDs - match_routes__begin_mid_end_points\n===')
linear_tmcs['rte_nm_match'] = linear_tmcs.apply(match_routes__begin_mid_end_points, axis=1)

# Remove null values.  These will need to be matched at the tmc level
linear_tmcs = linear_tmcs.loc[linear_tmcs['rte_nm_match'].notnull()].copy()

# Reduce to only needed columns to join back to the tmc dataframe
linear_tmcs = linear_tmcs.reset_index()[['TmcLinear', 'RoadName', 'rte_nm_match']].drop_duplicates()

# Remove any linear tmc id that appears more than once
linear_tmcs = linear_tmcs[~linear_tmcs.duplicated(['TmcLinear', 'RoadName'], keep=False)]

In [69]:
# Join tmcs and linear_tmcs by TmcLinear
tmcs = tmcs.merge(linear_tmcs, how='left', on=['TmcLinear', 'RoadName'])

In [70]:
print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

Remaining null values: 341


In [71]:
# Match by TMC, but only on tmcs where rte_nm_match is null
tmcs_no_match = tmcs.loc[tmcs['rte_nm_match'].isnull()].copy()

log.debug(f'===\TMCs - match_routes__begin_mid_end_points\n===')
tmcs_no_match['rte_nm_match'] = tmcs_no_match.apply(match_routes__begin_mid_end_points, axis=1)
tmcs_no_match = tmcs_no_match[['Tmc', 'rte_nm_match']]

# Join back to main tmcs
tmcs = tmcs.merge(tmcs_no_match, how='left', on='Tmc', suffixes=('', '_new')).fillna(tmcs_no_match)
tmcs.loc[tmcs['rte_nm_match'].isnull(), 'rte_nm_match'] = tmcs['rte_nm_match_new']
tmcs.drop(columns='rte_nm_match_new', axis=1, inplace=True)

print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

Remaining null values: 134


In [72]:
# Match by TMC again with a lower search radius, but only on tmcs where rte_nm_match is null
tmcs_no_match = tmcs.loc[tmcs['rte_nm_match'].isnull()].copy()

log.debug(f'===\TMCs - match_routes__begin_mid_end_points - low search radius\n===')
tmcs_no_match['rte_nm_match'] = tmcs_no_match.apply(match_routes__begin_mid_end_points, distance=5, axis=1)
tmcs_no_match = tmcs_no_match[['Tmc', 'rte_nm_match']]

# Join back to main tmcs
tmcs = tmcs.merge(tmcs_no_match, how='left', on='Tmc', suffixes=('', '_new')).fillna(tmcs_no_match)
tmcs.loc[tmcs['rte_nm_match'].isnull(), 'rte_nm_match'] = tmcs['rte_nm_match_new']
tmcs.drop(columns='rte_nm_match_new', axis=1, inplace=True)

print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

Remaining null values: 89


In [73]:
### Removed because it may be introducing errors ###

# # Match by TMC again with a higher search radius, but only on tmcs where rte_nm_match is null
# tmcs_no_match = tmcs.loc[tmcs['rte_nm_match'].isnull()].copy()

# log.debug(f'===\TMCs - match_routes__begin_mid_end_points - low search radius\n===')
# tmcs_no_match['rte_nm_match'] = tmcs_no_match.apply(match_routes__begin_mid_end_points, distance=25, axis=1)
# tmcs_no_match = tmcs_no_match[['Tmc', 'rte_nm_match']]

# # Join back to main tmcs
# tmcs = tmcs.merge(tmcs_no_match, how='left', on='Tmc', suffixes=('', '_new')).fillna(tmcs_no_match)
# tmcs.loc[tmcs['rte_nm_match'].isnull(), 'rte_nm_match'] = tmcs['rte_nm_match_new']
# tmcs.drop(columns='rte_nm_match_new', axis=1, inplace=True)

# print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

In [74]:
### Removed because it may be introducing errors ###

# # Match by TMC again, but using the overlap lrs instead
# tmcs_no_match = tmcs.loc[tmcs['rte_nm_match'].isnull()].copy()

# log.debug(f'===\TMCs - match_routes__begin_mid_end_points - overlap LRS\n===')
# tmcs_no_match['rte_nm_match'] = tmcs_no_match.apply(match_routes__begin_mid_end_points, distance=25, lrs=lrs_overlap, axis=1)
# tmcs_no_match = tmcs_no_match[['Tmc', 'rte_nm_match']]

# # Join back to main tmcs
# tmcs = tmcs.merge(tmcs_no_match, how='left', on='Tmc', suffixes=('', '_new')).fillna(tmcs_no_match)
# tmcs.loc[tmcs['rte_nm_match'].isnull(), 'rte_nm_match'] = tmcs['rte_nm_match_new']
# tmcs.drop(columns='rte_nm_match_new', axis=1, inplace=True)

# print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

In [75]:
def match_routes__line_segments(record, lrs, frequency=5, detailed_search=False, unfiltered=False):
    log.debug(f'\n{"="*10}')
    log.debug(f'\n{record["Tmc"]} - Match Routes by Segments')

    # Break input geometry into a list of individual line segments
    coords = record.geometry.coords
    segments = []
    for i in range(len(coords)):
        begin_point = coords[i-1]
        end_point = coords[i]
        segment = LineString((begin_point, end_point))
        segments.append(segment)

    if len(segments[1::frequency]) < frequency:
        frequency = 2  # Ensures that shorter segments get enough sample points

    if detailed_search:
        frequency = 1  # If detailed search, then all segments should be checked

    #  Reduce list to every {frequency} segment to improve processing time
    segments = segments[1::frequency]  # Ignore first segment because it contains begin and end point
    log.debug(f'\t{len(segments)} test segments')

    # For each segment, find matching nearby routes
    routes = []
    for segment in segments:
        nearby_routes = gp_lrs.get_nearby_route_by_segments(segment, 15, lrs, unfiltered=unfiltered)
        log.debug(f'\t\t{nearby_routes}')
        routes.extend(nearby_routes)
        
    # Check for no matches
    if len(routes) == 0:
        log.debug('\tNo matches.  Returning None')
        return None

    log.debug(f'\tAll Routes:  {routes}')

    # If only one segment tested and only one result, return the result
    if len(routes) == 1 and len(segments) == 1:
        return routes[0]

    # Keeps elements in a list that have a consecutive duplicate.
    route_list = []
    i = 0
    while i < len(routes) - 1:
        if routes[i] == routes[i + 1]:
            route_list.extend([routes[i], routes[i + 1]])
            i += 2  # Skip the next element since it's already included
        else:
            i += 1

    route_list = list(set(route_list))

    # If no matches found and this is not a detailed search, try again
    # but include all segments
    if len(route_list) == 0 and detailed_search == False:
        return match_routes__line_segments(record, lrs, detailed_search=True)

    log.debug(f'\tReturning Routes:  {route_list}')

    return ','.join(route_list)

In [76]:
null_filter = (tmcs['rte_nm_match'].isnull())
# null_filter = (tmcs['Tmc'] == '110N18081')
tmcs.loc[null_filter, 'rte_nm_match'] = tmcs.loc[null_filter].apply(match_routes__line_segments, lrs=lrs, axis=1)

print(f"Remaining null values: {len(tmcs.loc[tmcs['rte_nm_match'].isnull()])}")

Remaining null values: 2


In [77]:
# Records with more than one rte_nm in the rte_nm_match field should get new records

# Identify rows with multiple values
tmcs['multiple_values'] = tmcs['rte_nm_match'].str.contains(',')

# Explode the multiple values
tmcs_exploded = tmcs.assign(rte_nm_match=tmcs['rte_nm_match'].str.split(',')).explode('rte_nm_match')

In [78]:
# Reload overlap lrs without filter to find all required measures
lrs_overlap = gp_lrs.LRS(lrs_overlap_path)

In [79]:
# Get begin/end measure values
tmcs_m = gp_lrs.locate_features_along_routes(tmcs_exploded, 'Tmc', lrs, 'rte_nm_match')

In [80]:
# Flip routes so that non-prime segments will be drawn on non-prime LRS routes
round_1_output = gp_lrs.flip_routes(tmcs_m, 'RTE_NM', 'Begin_Msr', 'End_Msr', lrs_overlap)

# df_final_output will contain successful results of all rounds and will be updated at the
# end of each round below
df_final_output = round_1_output.copy()
df_final_output['round'] = 'Round 1'

In [81]:
def export_intermediate_results(geodataframe, csv_name, tbl_name, event_tbl_name, rte_nm_field="RTE_NM", begin_msr_field="Begin_Msr", end_msr_field="End_Msr"):
    # Export round 1 csv
    round_1_csv_path = os.path.join(output_path, csv_name)
    geodataframe.to_csv(round_1_csv_path, index=False)

    # Create round 1 event table
    tbl_round_1 = os.path.join(intermediate_gdb, tbl_name)
    arcpy.TableToTable_conversion(round_1_csv_path, intermediate_gdb, tbl_name)
    arcpy.DeleteField_management(tbl_round_1, 'geometry')

    # Create round 1 event layer
    round_1_results_path = os.path.join(intermediate_gdb, event_tbl_name)
    arcpy.lr.MakeRouteEventLayer(lrs_overlap_path, "RTE_NM", tbl_round_1, f"{rte_nm_field}; Line; {begin_msr_field}; {end_msr_field}", f"{event_tbl_name} Events", None, "NO_ERROR_FIELD", "NO_ANGLE_FIELD", "NORMAL", "ANGLE", "LEFT", "POINT")
    arcpy.conversion.FeatureClassToFeatureClass(f"{event_tbl_name} Events", intermediate_gdb, event_tbl_name)

export_intermediate_results(round_1_output, 'round_1.csv', 'tbl_round_1', 'round_1_results')

### Test hausdorff distance ###

In [82]:
log.debug(f'{"=" * 21}\n       ROUND 2       \n{"=" * 21}')
df_round_1_qc = gp.read_file(intermediate_gdb, layer='round_1_results')
df_round_1_qc = df_round_1_qc.to_crs(epsg=3968)
df_round_1_qc.head(3)

,Tmc,RoadNumber,RoadName,TmcLinear,begin_point,mid_point,end_point,RTE_NM,multiple_values,Begin_Msr,End_Msr,Shape_Length,geometry
0,110P17038,495.0,CAPITAL BELTWAY,1647,POINT (198036.71566126795 324753.0932277164),POINT (197967.9273798816 325026.60442200315),POINT (197926.78945732312 325305.5225492616),R-VA IS00495NBX,False,10.294,10.618,728.268579,"MULTILINESTRING ((198035.469 324752.614, 19801..."
1,110P17035,495.0,CAPITAL BELTWAY,1647,POINT (197867.3366734084 319164.24651148834),POINT (197859.85150635187 319432.751028379),POINT (197850.63153743424 319701.20193284395),R-VA IS00495NBX,False,6.706,7.048,692.747188,"MULTILINESTRING ((197869.975 319163.530, 19786..."
2,110P10960,620.0,BRADDOCK RD,949,POINT (203816.9247678481 317725.3749000386),POINT (203812.02527334285 317730.2467953629),POINT (203807.12577883757 317735.11869068723),R-VA029SC00620EB,False,22.284,22.290,17.720661,"MULTILINESTRING ((203817.028 317724.493, 20381..."


In [83]:
# Dissolve by TMC for QC so that TMCs with multiple records are checked as one single geometry
df_round_1_qc_dissolved = df_round_1_qc.dissolve(by='Tmc').reset_index()

In [84]:
# Get geometry of input tmcs to compare to output for QC
tmcs_geom_dict = tmcs[['Tmc', 'geometry']].set_index('Tmc').to_dict()['geometry']

In [85]:
def get_hausdorff_distance(record):
    hausdorff = shapely.hausdorff_distance(record.geometry, tmcs_geom_dict.get(record['Tmc']))

    confidence = 'Very High'
    if hausdorff > 5:
        confidence = 'High'
    if hausdorff > 20:
        confidence = 'Medium'
    if hausdorff > 75:
        confidence = 'Low'
    if hausdorff > 100:
        confidence = 'Very Low'
    
    if record['Begin_Msr'] == record['End_Msr'] or record['Shape_Length'] == 0:
        confidence = 'Very Low'
    
        
    record['hausdorff'] = hausdorff
    record['confidence'] = confidence
    
    return record


df_round_1_qc_dissolved = df_round_1_qc_dissolved.apply(get_hausdorff_distance, axis=1)

In [86]:
# Rerun tmcs with low and very low confidence values with detailed segment based location
low_confidence_tmcs_ids = df_round_1_qc_dissolved.loc[df_round_1_qc_dissolved['confidence'].isin(('Low', 'Very Low'))]['Tmc'].unique()
low_confidence_tmcs = round_1_output.loc[round_1_output['Tmc'].isin(low_confidence_tmcs_ids)].copy()

low_confidence_tmcs.drop(columns=('RTE_NM'), axis=1, inplace=True)
low_confidence_tmcs['Begin_Msr'] = None
low_confidence_tmcs['End_Msr'] = None

# Locate RTE_NM by segment
low_confidence_tmcs['rte_nm_match'] = low_confidence_tmcs.apply(match_routes__line_segments, lrs=lrs, detailed_search=True, axis=1)

# Identify rows with multiple values
low_confidence_tmcs['multiple_values'] = low_confidence_tmcs['rte_nm_match'].str.contains(',')

# Explode the multiple values
low_confidence_tmcs_exploded = low_confidence_tmcs.assign(rte_nm_match=low_confidence_tmcs['rte_nm_match'].str.split(',')).explode('rte_nm_match')

# Get begin/end measure values
low_confidence_tmcs_m = gp_lrs.locate_features_along_routes(low_confidence_tmcs_exploded, 'Tmc', lrs, 'rte_nm_match')

# Flip routes so that non-prime segments will be drawn on non-prime LRS routes
round_2_results = gp_lrs.flip_routes(low_confidence_tmcs_m, 'RTE_NM', 'Begin_Msr', 'End_Msr', lrs_overlap)

export_intermediate_results(round_2_results, 'round_2.csv', 'tbl_round_2', 'round_2_results')



In [87]:
# Append round 2 results to final output
round_2_tmcs = round_2_results['Tmc'].unique()
round_2_results['round'] = 'Round 2'
df_final_output = pd.concat((df_final_output.loc[~(df_final_output['Tmc'].isin(round_2_tmcs))], round_2_results))

In [88]:
# Rerun QC
df_round_2_qc = gp.read_file(intermediate_gdb, layer='round_2_results')
df_round_2_qc = df_round_2_qc.to_crs(epsg=3968)

# Dissolve by TMC for QC so that TMCs with multiple records are checked as one single geometry
df_round_2_qc_dissolved = df_round_2_qc.dissolve(by='Tmc').reset_index()

df_round_2_qc_dissolved = df_round_2_qc_dissolved.apply(get_hausdorff_distance, axis=1)

In [89]:
### TEST Round 3 ###
log.debug(f'{"=" * 21}\n       ROUND 3       \n{"=" * 21}')
# Rerun tmcs with low and very low confidence values with even more detailed segment based location if multiple_values == True

# Get list of TMCs with low confidence that are from multiple routes
low_confidence_tmcs_ids = df_round_2_qc_dissolved.loc[(df_round_2_qc_dissolved['confidence'].isin(('Medium', 'Low', 'Very Low')) & (df_round_2_qc_dissolved['multiple_values'] == 'True'))]['Tmc'].unique()

# Get fresh copy of source tmcs
original_tmcs = gp.read_file(tmcs_path)
original_tmcs = original_tmcs[['Tmc', 'RoadNumber', 'RoadName', 'TmcLinear', 'geometry']]
original_tmcs = original_tmcs.to_crs(epsg=3968)
low_confidence_tmcs = original_tmcs.loc[original_tmcs['Tmc'].isin(low_confidence_tmcs_ids)].copy()

# Locate RTE_NM by segment
low_confidence_tmcs['rte_nm_match'] = low_confidence_tmcs.apply(match_routes__line_segments, lrs=lrs, detailed_search=True, unfiltered=True, axis=1)

# Identify rows with multiple values
low_confidence_tmcs['multiple_values'] = low_confidence_tmcs['rte_nm_match'].str.contains(',')

# Get dictionary of routes that should be included with TMC ID as key and a list of RTE_NMs as values
low_confidence_tmcs_route = low_confidence_tmcs.groupby('Tmc')['rte_nm_match'].agg([('RTE_NM', ','.join)])
low_confidence_tmcs_route['RTE_NM'] = low_confidence_tmcs_route['RTE_NM'].str.split(',')
low_confidence_tmcs_route_dict = low_confidence_tmcs_route.to_dict()['RTE_NM']
for item in low_confidence_tmcs_route_dict:  # Remove duplicates
    low_confidence_tmcs_route_dict[item] = list(set(low_confidence_tmcs_route_dict[item]))

In [90]:
# For each segment in each mc in low_confidence_tmcs_ids, find the closest segment from the list of rte_nms
# and create an event record for that segment.


# Create events for Tmcs based on low_confidence_tmcs_route_dict
def match_routes__line_segments__with_m(record):
    log.debug(f'\n{"="*10}')
    log.debug(f'\n{record["Tmc"]} - Match Routes by Segments with m-values')
    rte_nms = low_confidence_tmcs_route_dict[record['Tmc']]
    log.debug(f'\tFrom RTE_NMs: {rte_nms}')

    # Break input geometry into a list of individual line segments
    coords = record.geometry.coords
    segments = []
    for i in range(len(coords)):
        begin_point = coords[i-1]
        end_point = coords[i]
        segment = LineString((begin_point, end_point))
        segments.append(segment)
    
    def get_nearest_route_segment(segment, rte_nm):
        lrs_route = lrs.geodataframe_unfiltered.loc[lrs.geodataframe_unfiltered['RTE_NM'] == rte_nm].copy()
        point = segment.centroid

        if len(lrs_route) == 0:
            return None

        if len(lrs_route) > 1:
            # Find distance of each segment to the input point
            # Return segment with minimum distance
            lrs_route['dist'] = lrs_route.geometry.distance(point)
            lrs_route = lrs_route.loc[lrs_route['dist'] == lrs_route['dist'].min()]

        lrs_route = lrs_route.iloc[0]
        route_geom = lrs_route.geometry  # Geometry of lrs route

        ranked_segments = []
        for i in range(len(route_geom.coords) - 1):
            route_segment = LineString([route_geom.coords[i], route_geom.coords[i+1]])
            segment_distance = route_segment.distance(point)
            ranked_segments.append((route_segment, segment_distance))
        ranked_segments.sort(key=lambda x: x[1])  # Sort by segment_distance

        return (rte_nm, ranked_segments[0][0], ranked_segments[0][1])

    segments_to_keep = []
    for segment in segments[1:]:
        log.debug(f'\t\tTMC Segment: {segment}')
        potential_segments = []
        for rte_nm in rte_nms:
            potential_segments.append(get_nearest_route_segment(segment, rte_nm))
        
        # print(potential_segments[0])
        potential_segments.sort(key=lambda x: x[-1])
        log.debug(f'\t\t\tKeeping lrs segment {potential_segments[0][:2]}')
        segments_to_keep.append(potential_segments[0][:2])
    segments_to_keep = list(set(segments_to_keep))
    
    # Get m values for segments to keep
    segments_to_keep_m = []
    for segment in segments_to_keep:
        begin_msr, end_msr = gp_lrs.get_segment_msr(segment[1], lrs, segment[0])
        segments_to_keep_m.append({
            'Tmc': record['Tmc'],
            'RTE_NM': segment[0],
            'Begin_Msr': begin_msr,
            'End_Msr': end_msr
        })

    low_confidence_segments.extend(segments_to_keep_m) 

low_confidence_segments = []
low_confidence_tmcs.apply(match_routes__line_segments__with_m, axis=1)

# Create DataFrame with Round 3 output
round_3_preflip = pd.DataFrame(low_confidence_segments)

# Dissolve output
round_3_preflip = round_3_preflip.groupby(['Tmc', 'RTE_NM']).agg({'Begin_Msr': 'min', 'End_Msr': 'max'}).reset_index()


In [91]:
# Round 3's methodology makes it impossible to flip the routes in the same was as above because the directionality
# of the output geometry does not necessarily match the directionality of the input.  A new method is used in this cell

# Create intermediate event layer to get Round 3 output geometry
export_intermediate_results(round_3_preflip, 'round_3_preflip.csv', 'tbl_round_3_preflip', 'round_3_preflip')
round_3_preflip_path = r'C:\Users\daniel.fourquet\Documents\Tasks\TMC Conflation 2025\NPMRDS\intermediate.gdb\round_3_preflip'

# Open as geodataframe
round_3_preflip = gp.read_file(intermediate_gdb, layer='round_3_preflip')
round_3_preflip = round_3_preflip.to_crs(epsg=3968)

In [92]:
# For each record, find the distance between the first point in the input TMC geometry and the first and last points of
# the output geometry.  The point with the shorter distance should be associated with the Begin_Msr.  If the end point
# in the output geometry is closer to the begin point of the input geometry than the begin point in the output geometry,
# then the msr values should be switched.

def round_3__identify_segments_to_flip(record):
    tmc = record['Tmc']
    begin_msr = record['Begin_Msr']
    end_msr = record['End_Msr']
    log.debug('\n==========\n')
    log.debug(f'{tmc} - round 3 flip')
    log.debug(f'\tInput RTE_NM: {record["RTE_NM"]}')

    if record['RTE_NM'].startswith('D-'):
        # Cannot flip.  This will break the function anyway.
        return record


    # Get input tmc geometry
    tmc_geom = tmcs_geom_dict.get(tmc)
    tmc_geom_first_point = shapely.get_point(tmc_geom, 0)
    log.debug(f'\tTMC first point: {tmc_geom_first_point}')

    # Get round 3 output geometry
    round_3_geom = record.geometry
    if isinstance(round_3_geom, MultiLineString):
        round_3_first_point = shapely.get_point(round_3_geom.geoms[0], 0)
        round_3_last_point = shapely.get_point(round_3_geom.geoms[-1], -1)
    else:
        round_3_first_point = shapely.get_point(round_3_geom, 0)
        round_3_last_point = shapely.get_point(round_3_geom, -1)

    log.debug(f'\tRound 3 first point: {round_3_first_point}')
    log.debug(f'\tRound 3 last point: {round_3_last_point}')

    # Measure distance from round 3 points to input point
    round_3_first_point_dist = shapely.distance(tmc_geom_first_point, round_3_first_point)
    round_3_last_point_dist = shapely.distance(tmc_geom_first_point, round_3_last_point)
    log.debug(f'\n\tRound 3 first point distance: {round_3_first_point_dist}')
    log.debug(f'\tRound 3 last point distance: {round_3_last_point_dist}')

    to_flip = round_3_last_point_dist < round_3_first_point_dist
    log.debug(f'\n\tTo flip?: {to_flip}')
    record['to_flip'] = to_flip

    # If the above method determines a flip is needed, set the begin measure to be
    # the higher of the two measures so that the flip_routes function will flip
    # the segment
    record['ori_RTE_NM'] = record['RTE_NM']
    record['ori_Begin_Msr'] = record['Begin_Msr']
    record['ori_End_Msr'] = record['End_Msr']
    if to_flip:

        record['Begin_Msr'] = max(begin_msr, end_msr)
        record['End_Msr'] = min(begin_msr, end_msr)

    return record
    

round_3_results = round_3_preflip.apply(round_3__identify_segments_to_flip, axis=1)
round_3_results.loc[round_3_results['to_flip'] == True] = gp_lrs.flip_routes(round_3_results.loc[round_3_results['to_flip'] == True], rte_nm='RTE_NM', begin_msr='Begin_Msr', end_msr='End_Msr', lrs=lrs)

# Export round 3 results
round_3_results['round'] = 'Round 3'
export_intermediate_results(round_3_results, 'round_3_results.csv', 'tbl_round_3_results', 'round_3_results')

IndexError: index out of range

In [93]:
round_3_results 

,Tmc,RTE_NM,Begin_Msr,End_Msr,Shape_Length,geometry,to_flip,ori_RTE_NM,ori_Begin_Msr,ori_End_Msr,round
0,110P19585,R-VA US00013NB,17.460,18.010,631.500574,"MULTILINESTRING ((256275.831 84407.624, 256252...",False,R-VA US00013NB,17.460,18.010,Round 3
1,110P09359,R-VA US00058EBBUS007,0.192,0.367,375.833044,"MULTILINESTRING ((308315.327 99323.160, 308320...",False,R-VA US00058EBBUS007,0.192,0.367,Round 3
2,110P09359,R-VA US00058EBBUS007RMP000.00A,0.000,0.100,269.000759,"MULTILINESTRING ((308359.850 99340.500, 308394...",False,R-VA US00058EBBUS007RMP000.00A,0.000,0.100,Round 3
3,110P04909,R-VA US00017NB RMP017.00C,0.000,0.280,619.926875,"MULTILINESTRING ((288453.446 88841.747, 288469...",False,R-VA US00017NB RMP017.00C,0.000,0.280,Round 3
4,110N08784,R-VA US00013SB,49.615,49.492,284.304503,"MULTILINESTRING ((293202.724 102069.670, 29320...",True,R-VA US00013SB,49.492,49.615,Round 3
5,110P09359,R-VA US00058EB,506.060,506.084,89.223691,"MULTILINESTRING ((308563.457 99395.551, 308583...",False,R-VA US00058EB,506.060,506.084,Round 3
6,110N08784,R-VA SR00165SB,32.644,32.605,88.847374,"MULTILINESTRING ((293212.597 102309.402, 29321...",True,R-VA SR00165SB,32.605,32.644,Round 3
7,110P04909,R-VA IS00464NB RMP001.00A,0.062,0.267,449.021061,"MULTILINESTRING ((288657.756 89249.381, 288655...",False,R-VA IS00464NB RMP001.00A,0.062,0.267,Round 3
8,110N04915,R-VA IS00464SB,5.830,5.413,1083.230418,"MULTILINESTRING ((286619.118 97208.591, 286616...",True,R-VA IS00464SB,5.413,5.830,Round 3
9,110N04909,R-VA US00017SB,16.690,16.459,488.891886,"MULTILINESTRING ((287902.959 88584.114, 287981...",True,R-VA US00017SB,16.459,16.690,Round 3


In [ ]:
# Append round 2 results to final output
round_3_tmcs = round_3_results['Tmc'].unique()
df_final_output = pd.concat((df_final_output.loc[~(df_final_output['Tmc'].isin(round_3_tmcs))], round_3_results))

# Ensure all segments are properly flipped:
df_final_output_flip = gp_lrs.flip_routes(df_final_output, rte_nm='RTE_NM', begin_msr='Begin_Msr', end_msr='End_Msr', lrs=lrs)

# Clean up df_final_output
final_output_fields = ['Tmc', 'RTE_NM', 'Begin_Msr', 'End_Msr', 'round']
df_final_output_flip = df_final_output_flip[final_output_fields]

# Export pre-qc df_final_output
export_intermediate_results(df_final_output_flip, 'final_output.csv', 'tbl_final_output', 'final_output')

In [ ]:
# Rerun QC
df_final_qc = gp.read_file(intermediate_gdb, layer='final_output')
df_final_qc = df_final_qc.to_crs(epsg=3968)

# Dissolve by TMC for QC so that TMCs with multiple records are checked as one single geometry
df_final_qc_dissolved = df_final_qc.dissolve(by='Tmc').reset_index()

df_final_qc_dissolved = df_final_qc_dissolved.apply(get_hausdorff_distance, axis=1)

df_final_qc_dissolved.to_clipboard(index=False)